## Scripts for Automatic computing timeseries densities of DKIST Cryo-NISRP or DL-NIRSP data

The dkist_dens function needs a chianti help file which is attached.

These scripts are not fully functional and free of bugs or improper usage, but more of a proof of concept.

Contact: Alin Paraschiv, NSO arparaschiv at nso.edu

In [1]:
def DKIST_DENS(i1074,i1079,chianti_path,index):
## compute the density from two stokes I observations (full arrays) of Fe XIII assuming a ratio table provided by chianti and that the arrays are aligned properly.

## - The chianti table is computed with IDL and imported via the path given to this function.
## - Requires the observation index from one of the datasets
## - requires the numpy, scipy interpolate and io.readsav imports ## asusmed to be loaded by the calling function
    import numpy as np
    from scipy.io import readsav
    from scipy import interpolate

    ## Get sizes of arrays
    nx     = i1074.shape[0]
    if  (len(i1074.shape) != 2 ):  
        ny     = 1   
        prin("1D-only slit data is used")
    else:    
        ny     = i1074.shape[1] 
    
    if (i1074.shape != i1079.shape):   ## sanity check for input arrays
        print("Arrays are not of the same shape or not 2D. Aborting!")
        return np.zeros((nx,ny),dtype=np.float64)

    rat_obs=i1074/i1079 #the observed intensity ratio

    ## theoretical chianti ratio calculations done with IDL following instruction in the preamble
    chia=readsav(chianti_path)
    #chia.keys()  ##    # check the arrays
    ##  in the save file, 'h' is an array of heights in solar radii (from 1.05 to 1.40); 
    ##  'den' is an array of density (from 5.0 to 13.0), 'rat' is a 2D array containing line ratios corresponding to different density values at different heights.
    ##  To query the database of heights:
    ##  print(chia.h,chia.h.shape,chia.rat.shape,chia.den.shape)
    ##  h= [1.        1.01      1.02      1.03      1.04      1.05      1.06
    ##      1.07      1.08      1.09      1.1       1.11      1.12      1.13
    ##      1.14      1.15      1.16      1.17      1.18      1.19      1.2
    ##      1.21      1.22      1.23      1.24      1.25      1.26      1.27
    ##      1.28      1.29      1.3       1.31      1.3199999 1.3299999 1.34
    ##      1.35      1.36      1.37      1.38      1.39      1.4       1.41
    ##      1.42      1.43      1.44      1.45      1.46      1.47      1.48     1.49] 
    ## h is of shape [50]; den and rat are arrays of shape [50, 33] corresponding to the 50 h heights above the limb


    density=np.zeros((nx,ny),dtype=np.float64)
    for xx in np.arange(0,nx):
        for yy in np.arange(0,ny):
            if (np.isnan(rat_obs[xx,yy]) or np.isinf(rat_obs[xx,yy])):      ## discard nans and infs
                density[xx,yy] = 0
            else:                                                           ## main loop
                rpos = np.sqrt((crval2+cdelt2*xx)**2+(crval3+yy*cdelt3)**2)/961
                subh = np.argwhere(chia.h > rpos)                                                                                      ## find the corresponding height (in solar radii) for each pixel
                #print(len(subh),rpos/rsun)                      
                if len(subh) == 0: subh=[-1]                                                                                                ## if height is greater than maximum h (1.5R_sun) just use the 1.5R_sun corresponding ratios
                ifunc = interpolate.interp1d(chia.rat[subh[0],:].flatten(),chia.den[subh[0],:], kind="quadratic",fill_value="extrapolate")  ## make the interpolation function; Quadratic as radial density drop is usually not linear
                density[xx,yy] = ifunc(rat_obs[xx,yy])                                                                                      ## apply the interpolation to the data 
            else:                                                       ## values are "0" outside of the main section defined by the radiuses                                                                                  
                rat_obs[xx,yy]=0
                density[xx,yy]=0

    return density

#### Now, one would only compute the density for observation pair of Fe XIII 1074 and 1079 observations.

This can of course be included in a for loop to cycle through all frames.
Takes a couple of seconds to execute.

In [ ]:

dens_rat=COMP_DENS(stokes_i_1074,stokes_i_1079,'./chianti_v10_fe13_h50_ratio.sav',c1074_iqu['index1'])

plt.figure(figsize=(18,18))
plt.imshow(np.log10(dens_rat),vmin=7,vmax=8.5,cmap='Blues',origin='lower')
plt.colorbar(label='Log N$_e$')